# Web Scraping

In [163]:
import requests
import sys
import re
from bs4 import BeautifulSoup as bs
import pandas as pd, numpy as np
import warnings
warnings.filterwarnings(action='once')
from icecream import ic
from random import randint, random
from time import sleep
from timeit import default_timer as timer
from playsound import playsound

In [193]:
url = "https://www.metacritic.com"
# first_page = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed"
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0'}
# review_dict = {'name':[], 'date':[], 'game':[], 'rating':[], 'review':[]}
# game_dict = {'game':[], 'release_date':[], 'genre':[], 'platforms':[], 'developer':[], 'esrb_rating':[], 'ESRBs':[], \
#              'metascore':[], 'userscore':[],'critic_reviews':[], 'user_reviews':[], 'num_players':[], 'summary':[] }

In [186]:
def get_details(soup, text, col, idx=1):
    details = []
    if soup.find('th', scope='row', text = text)==None:
        game_dict[col].append(None)
        return
    for i in soup.find('th', scope='row', text = text).next_siblings:
        details.append(i.text)
    details_ = ' '.join(details[idx].strip().split())
    game_dict[col].append(details_)

In [187]:
def get_platforms(soup):
    if soup.find('span', class_="label", string = "Also On:")==None:
        game_dict['platforms'].append(soup.find('span', class_="platform").text.strip())
    else:
        consoles = [soup.find('span', class_="platform").text.strip()]
        other_consoles = [console.text  for console in soup.find_all('a', class_="hover_none")[1:]]
        consoles.extend(other_consoles)
        game_dict['platforms'].append(', '.join(consoles))

In [188]:
def get_user_reviews(soup):
    if soup.find_all('span', class_ = 'count')[1].find('a')!=None:
        game_dict['user_reviews'].append(soup.find_all('span', class_ = 'count')[1].find('a').text[:-8])
    else:
        game_dict['user_reviews'].append(None)

In [189]:
def get_numplayers(soup):
    P = soup.find(text = 'Number of Players:')
    O = soup.find(text = 'Number of Online Players:')
    if P==None:
        if O==None or O.next_element.text=='No Online Multiplayer':
            game_dict['num_players'].append('1 Player')
            return
        else:
            game_dict['num_players'].append(O.next_element.text)
            return
    else:
        if O==None or O.next_element.text=='No Online Multiplayer':
            game_dict['num_players'].append(P.next_element.text)
            return
        else:
            players = [P.next_element.text, O.next_element.text]
            game_dict['num_players'].append(', '.join(players))
            return

In [225]:
def get_summary(soup):
    if soup.find('span', class_="label", string = "Also On:")!=None:
        if len(soup.find_all('span', class_ = 'data'))==4:
            game_dict['summary'].append(soup.find_all('span', class_ = 'data')[3].text)
        else:
            game_dict['summary'].append(None)
    else:
        if len(soup.find_all('span', class_ = 'data'))==3:
            game_dict['summary'].append(soup.find_all('span', class_ = 'data')[2].text)
        else:
            game_dict['summary'].append(None)

In [191]:
def parse_info(details_soup):
    # Get to Details Page
    
#     testing
#     if soup.find('a', attrs = {'class':'action'}, text = 'Details & Credits')==None:
#         print('SOUP')
#         print(soup)
#         return 'Done'
#     print(soup.find('a', attrs = {'class':'action'}, text = 'Details & Credits'))
#     return
    
#     details_link = url + soup.find('a', attrs = {'class':'action'}, text = 'Details & Credits').get('href')
#     details_response = requests.get(details_link, headers = user_agent)
#     details_soup = bs(details_response.text, 'html.parser')
    
    count = 1 
    while details_soup.a==None and count<3:
        print('Error, Trying DETAILS Again')
        sleep(3)
        details_response = requests.get(details_link, headers = user_agent)
        details_soup = bs(details_response.text, 'html.parser')
        count += 1
        print('Still None?:', details_soup.a==None)
        print('response code:', details_response.status_code)
    
#     Get game Details
    try:
        for detail in details_soup.find_all('div', id='main'):
            game_dict['title'].append(detail.find('h1').text)
            game_dict['release_date'].append(detail.find_all('span', class_='data')[1].text)
            get_details(detail, 'Genre(s):', 'genre')
    #         get_platforms(detail)
            game_dict['platforms'].append(detail.find('span', class_="platform").text.strip())
            get_details(detail, 'Developer:', 'developer')
            get_details(detail, 'Rating:', 'esrb_rating')
            get_details(detail, 'ESRB Descriptors:', 'ESRBs', idx=0)
            game_dict['metascore'].append(detail.find('span', itemprop='ratingValue').text)
            game_dict['userscore'].append(detail.find_all('a', class_='metascore_anchor')[1].contents[1].text)
            game_dict['critic_reviews'].append([i.find_all('span') for i in 
                                                detail.find_all('span', class_='count')][0][2].text.strip())
            get_user_reviews(detail)
            get_numplayers(detail)
            get_summary(detail)
    except Exception as e:
        print(sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print('Info Failed')
#         playsound('ding.mp3')
        return 'done'

In [179]:
def parse_reviews(user_review_soup):
    #  Extracts link to User Reviews Page from main game page
#     for user_review in soup.find_all('a', attrs = {'class':'action'}, text = 'User Reviews'):
#         user_review_link = url + user_review.get('href')

# #             Connects to User Review page
#         user_review_response = requests.get(user_review_link, headers = user_agent)
#         user_review_soup = bs(user_review_response.text, 'html.parser')
        
        
        
    count = 1
    while user_review_soup.a==None and count<3:
        print('Error, Trying REVIEWS Again')
        sleep(3)
        user_review_response = requests.get(user_review_link+'/robots.text', headers = user_agent)
        user_review_soup = bs(user_review_response.text, 'html.parser')
        count += 1
        print('Still None?:', user_review_soup.a==None)
        print('response code:', user_review_response.status_code)

#           credit: Adeline Ong
#           https://towardsdatascience.com/web-scraping-metacritic-reviews-using-beautifulsoup-63801bbe200e

#             Loop through User Review Page and Extract Review Data into a dictionary
    try:
        for review in user_review_soup.find_all('div', class_='review_content'):
            if review.find('div', class_='name') == None:
                break
    #                 get_consoles(user_review_soup) 
            review_dict['game'].append(user_review_soup.find('h1').text)
            review_dict['name'].append(review.find('div', class_='name').find(recursive = True).text)
            review_dict['date'].append(review.find('div', class_='date').text)
            review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
            if review.find('span', class_='blurb blurb_expanded'):
                review_dict['review'].append(review.find('span', class_='blurb blurb_expanded').text)
            elif review.find('div', class_='review_body').find('span')==None:
                review_dict['review'].append(None)
            else:
                review_dict['review'].append(review.find('div', class_='review_body').find('span').text)
    except Exception as e:
        print(sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print('Reviews Failed')
#         playsound('ding.mp3')
        return 'done'

In [231]:
def parse_games(startpage_link, next_page, end_page, game_num_start, game_num_end = 101):  
    next_page = next_page
    
#   request from current page
    try:
        response = requests.get(startpage_link, headers = user_agent)
        soup = bs(response.text, 'html.parser')
        current_page = int(soup.find('span', class_='page_num').text)
        print(f'parsing page: {current_page}/{end_page}')
    except Exception as e:
        print(sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
        print(f'Cannot Connect to page {next_page-1}')
#         playsound('ding.mp3')
        return
    
#     Loop through current page and extract links to game pages
    game_num = game_num_start
    for game in soup.find_all('a', attrs = {'class':'title'})[game_num - 1:game_num_end]:
        game_link = url + game.get('href')
        
#         Connects to game page summary
        sleep(1)
        try:
            game_response = requests.get(game_link, headers = user_agent)
            game_soup = bs(game_response.text, 'html.parser')
        except Exception as e:
            print(sys.exc_info()[0],'|||', sys.exc_info()[1], '||| Error on line', sys.exc_info()[2].tb_lineno)
            print('Failing to connect to game page')
#             playsound('ding.mp3')
            
        
#         testing
        
        
        count = 1
        while game_soup.a==None and count<2:
            print('Error, Trying GAME Again')
            sleep(3)
            game_response = requests.get(game_link, headers = user_agent)
            game_soup = bs(game_response.text, 'html.parser')
            count += 1
            print('Still None?:', game_soup.a==None)
            print('response code:', game_response.status_code)
        
            if game_soup.a==None:
                sleep(3)
                print('Trying Last Time')
                game_response = requests.get(game_link+'/robots.text', headers = user_agent)
                game_soup = bs(details_response.text, 'html.parser')
                print('Still None?:', game_soup.a==None)
                print('response code:', game_response.status_code)
                if game_soup.find('a')==None:
                    print(f'Game {game_num} GAME Failed')
    #                 playsound('ding.mp3')
                    return
        
        sleep(3)
        
        
        details_link = url + game_soup.find('a', attrs = {'class':'action'}, text = 'Details & Credits').get('href')
#         Connect to Details Page
        details_response = requests.get(details_link, headers = user_agent)
        details_soup = bs(details_response.text, 'html.parser')

        if details_soup.find('a')==None:
            details_response = requests.get(details_link+'/robots.text', headers = user_agent)
            details_soup = bs(details_response.text, 'html.parser')
            if details_soup.find('a')==None:
                print(f'Game {game_num} Details Failed')
#                 playsound('ding.mp3')
                return
        
        for user_review in game_soup.find_all('a', attrs = {'class':'action'}, text = 'User Reviews'):
            user_review_link = url + user_review.get('href')

    #             Connects to User Review page
            user_review_response = requests.get(user_review_link, headers = user_agent)
            user_review_soup = bs(user_review_response.text, 'html.parser')
            if user_review_soup.a==None:
                user_review_response = requests.get(user_review_link+'/robots.text', headers = user_agent)
                user_review_soup = bs(user_review_response.text, 'html.parser')
                if user_review_soup.a==None:
                    print(f'Game {game_num} Reviews Failed')
#                     playsound('ding.mp3')
                    return
        
        
#         Get Game info
        if parse_info(details_soup)=='done':
            return
        
        
#         Get Game Reviews
        if parse_reviews(user_review_soup)=='done':
            return
        
        print(f'Game {game_num} completed')
        game_num +=1
        
    if next_page == (end_page+1):
        find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
        startpage_link = url + find_next_page.get('href')
        print(f'Done, Next Start Page: {startpage_link}')
        print('Copy and Paste this link or use startpage_link variable')
#         playsound('ding.mp3')
        return startpage_link
    if next_page>(end_page+1):
        print('Increase End page')
#         playsound('ding.mp3')
        return
    if current_page>=next_page:
        print('Increase Next page')
#         playsound('ding.mp3')
        return
    sleep(10)
    
#     Recurse through the following pages
    find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
    nextpage_link = url + find_next_page.get('href')
    next_page += 1
    parse_games(nextpage_link, next_page, end_page, game_num_start = 1, game_num_end = 101)

In [232]:
startpage='https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed&page=16'

In [ ]:
# start = timer()
# parse_games(startpage, next_page = 18, end_page = 20, game_num_start = 50, game_num_end = 101)
# end = timer()
# minutes = (end-start)//60
# seconds = int((end-start)-(minutes*60))
# print(f'Elapsed Time: {minutes} minutes {seconds} seconds')

In [229]:
# start = timer()
# parse_games(startpage, next_page = 16, end_page = 20, game_num_start = 99, game_num_end = 101)
# end = timer()
# minutes = (end-start)//60
# seconds = int((end-start)-(minutes*60))
# print(f'Elapsed Time: {minutes} minutes {seconds} seconds')

parsing page: 15/20
Game 99 completed
Game 100 completed
parsing page: 16/20
Game 1 completed
Game 2 completed
Game 3 completed
Game 4 completed
Game 5 completed
Game 6 completed
Game 7 completed
Game 8 completed
Game 9 completed
Game 10 completed
Game 11 completed
Game 12 completed
Game 13 completed
Game 14 completed
Game 15 completed
Game 16 completed
Game 17 completed
Game 18 completed
Game 19 completed
Game 20 completed
Game 21 completed
Game 22 completed
Game 23 completed
Game 24 completed
Game 25 completed
Game 26 completed
Game 27 completed
Game 28 completed
Game 29 completed
Game 30 completed
Game 31 completed
Game 32 completed
Game 33 completed
Game 34 completed
Game 35 completed
Game 36 completed
Game 37 completed
Game 38 completed
Game 39 completed
Game 40 completed
Game 41 completed
Game 42 completed
Game 43 completed
Game 44 completed
Game 45 completed
Game 46 completed
Game 47 completed
Game 48 completed
Game 49 completed
Game 50 completed
Game 51 completed
Game 52 comple

AttributeError: 'NoneType' object has no attribute 'get'

In [373]:
num_games['title'][169], num_games['num_games'][169]

('Test Drive Le Mans', 3)

In [372]:
# num_games['num_games'].insert()

In [57]:
# game_dict['summary'].append(None)

In [56]:
mylist = [1,2,3,4,5]
mylist.append(8)
mylist

[1, 2, 3, 4, 5, 8]

In [319]:
'title' in list(dir())

False

In [336]:
num_games['title'][-1]

"Sid Meier's Civilization II"

In [322]:
num_games = {'title':[], 'num_games':[]}
num_games['num_games'].append(int(soup.find_all('p', class_='see_all')[1].text.split()[2]))

1557

In [196]:
# link = 'https://www.metacritic.com/game/dreamcast/nfl-2k1/details'
# response = requests.get(link, headers = user_agent)
# soup = bs(response.text, 'html.parser')

In [204]:
# soup.find('span', class_="platform").text.strip()

In [ ]:
# ['game', 'release_date', 'genre', 'platforms', 'developer', 'esrb_rating', 'ESRBs', 'metascore', 'userscore', 
#  'critic_reviews', 'user_reviews', 'num_players', 'summary'])

In [233]:
games_df = pd.DataFrame(game_dict)
games_df

,title,release_date,genre,platforms,developer,esrb_rating,ESRBs,metascore,userscore,critic_reviews,user_reviews,num_players,summary
0,The Legend of Zelda: Ocarina of Time,"Nov 23, 1998","Action Adventure, Fantasy",Nintendo 64,Nintendo,E,Violence,99,9.1,22,6982.0,1 Player,"As a young boy, Link is tricked by Ganondorf, ..."
1,Tony Hawk's Pro Skater 2,"Sep 20, 2000","Sports, Alternative, Skateboarding",PlayStation,Neversoft Entertainment,T,Mild Animated Violence Mild Language,98,7.4,19,786.0,1-2 Players,As most major publishers' development efforts ...
2,Grand Theft Auto IV,"Apr 29, 2008","Action Adventure, Modern, Modern, Open-World",PlayStation 3,Rockstar North,M,Blood Intense Violence Partial Nudity Strong L...,98,7.8,64,4529.0,1 Player,[Metacritic's 2008 PS3 Game of the Year; Also ...
3,SoulCalibur,"Sep 8, 1999","Action, Fighting, 3D",Dreamcast,Namco,T,Animated Violence Suggestive Themes,98,8.5,24,373.0,1-2 Players,"This is a tale of souls and swords, transcendi..."
4,Grand Theft Auto IV,"Apr 29, 2008","Action Adventure, Modern, Modern, Open-World",Xbox 360,Rockstar North,M,Blood Intense Violence Partial Nudity Strong L...,98,7.9,86,3594.0,1 Player,[Metacritic's 2008 Xbox 360 Game of the Year; ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1647,Tom Clancy's Splinter Cell Double Agent,"Oct 17, 2006","Action Adventure, Modern, General, Modern",Xbox 360,Ubisoft Shanghai,M,Blood Language Violence,85,8.1,59,174,"1-6 Players, 6 Players Online Up to 6 Players",Veteran agent Sam Fisher is back. But he's nev...
1648,Pillars of Eternity: Complete Edition,"Aug 29, 2017","Role-Playing, Western-Style",PlayStation 4,"Obsidian Entertainment, Paradox Arctic",M,None,85,7.1,25,122,1 Player,"Pillars of Eternity, the ultimate role-playing..."
1649,Bravely Default,"Feb 7, 2014","Console-style RPG, Role-Playing, Console-style...",3DS,Silicon Studio,T,None,85,8.4,67,683,1 Player,[Available in the U.S. - Feb 2014] This new ye...
1650,Guacamelee! 2,"Jan 18, 2019","Action, Platformer, 2D,",Xbox One,DrinkBox Studios,None,None,85,6.6,9,32,Up to 4 Players,Brawling has never been more beautiful! The lo...


In [235]:
# games_df.to_csv('metacritic_games_four.csv')

In [162]:
# game_dict = pd.read_csv('metacritic_games_two.csv').drop(['Unnamed: 0'],axis=1).to_dict('list')

In [234]:
reviews_df= pd.DataFrame(review_dict)
reviews_df

,name,date,game,rating,review
0,ThomasR.,"Apr 3, 2003",Combat Mission: Barbarossa to Berlin,9,Needs a brush-up on the graphics.
1,JoaquinD.,"Aug 25, 2003",Combat Mission: Barbarossa to Berlin,10,Perfect strategic and tactical WWII wargame. ...
2,MichaelDorosh,"Aug 19, 2011",Combat Mission: Barbarossa to Berlin,10,"Still the gold standard for WEGO, squad-based,..."
3,dinin70,"May 3, 2017",Combat Mission: Barbarossa to Berlin,9,I don't think it deserves 10/10 for a simple r...
4,BlueFalcon,"Apr 9, 2005",NHL 2002,6,"I was hyped about this release at the time, b..."
...,...,...,...,...,...
49852,marc68,"May 15, 2016",DOOM,8,"This game is total quality. Campaign is great,..."
49853,silversun101,"May 15, 2016",DOOM,9,An utterly brilliant homage to the old school ...
49854,Mamal,"May 15, 2016",DOOM,8,Pros:\r +Fast paced combat\r+Great graphics ru...
49855,ascedentevil,"May 18, 2016",DOOM,9,A throwback to the old with pulse pounding act...


In [236]:
# reviews_df.to_csv('metacritic_reviews_four.csv')

In [216]:
reviews = reviews_df.sort_values('name')
reviews['ids'] = reviews.groupby('name').ngroup()
check_uniques = reviews[['ids','name','game','rating','review']]

check_uniques[['ids']].value_counts()[check_uniques[['ids']].value_counts()>2]

ids  
20981    94
1140     87
2605     56
17978    45
23012    43
         ..
1037      3
9         3
1203      3
1034      3
1781      3
Length: 1974, dtype: int64

In [227]:
for key in list(game_dict.keys())[:]:
    print(key,'length is', len(game_dict[key]), end = ", ")
#     newlist = game_dict[key][:-3]
#     print(f'New {key}','length is', len(newlist))
#     game_dict[key] = newlist

title length is 1501, release_date length is 1501, genre length is 1501, platforms length is 1501, developer length is 1501, esrb_rating length is 1501, ESRBs length is 1501, metascore length is 1501, userscore length is 1501, critic_reviews length is 1501, user_reviews length is 1501, num_players length is 1501, summary length is 1501, 

In [228]:
for key in list(review_dict.keys())[:]:
    print(key,'length is', len(review_dict[key]), end = ", ")
#     newlist = review_dict[key][:-1]
#     print(f'New {key}','length is', len(newlist))
#     review_dict[key] = newlist

name length is 43529, date length is 43529, game length is 43529, rating length is 43529, review length is 43529, 

In [304]:
# for i,v in enumerate(review_dict['name']): 
#         print(i,v)

In [423]:
# link = 'https://www.metacritic.com/game/dreamcast/test-drive-le-mans'
# link ='https://www.metacritic.com/game/xbox-360/rock-band-3'
link = 'https://www.metacritic.com/game/pc/sid-meiers-civilization-ii'
response = requests.get(link, headers = user_agent)
soup = bs(response.text, 'html.parser')

In [408]:
for review_num in soup.find_all('p', class_='see_all'):
    if review_num.find(href=re.compile('user-reviews')) not in soup.find_all('p', class_='see_all'):
        print(int(len(soup.find('ol', class_= 'reviews user_reviews').find_all('div', class_='review_body'))))
    else:
        print(int(soup.find_all('p', class_='see_all')[1].text.split()[2]))
        
#     i.find(href=re.compile('critic-reviews'))
#     int(game_soup.find_all('p', class_='see_all')[1].text.split()[2])

7
7


In [418]:
item_count = len(soup.find_all('p', class_='see_all'))
if item_count==2:
    print(int(soup.find_all('p', class_='see_all')[1].text.split()[2]))
elif item_count==1:
    if soup.find('p', class_='see_all').find(href=re.compile('user'))::
#         append text
#     else:
#         count reviews
# else:
#     count reviews

25


In [426]:
if soup.find('p', class_='see_all').find(href=re.compile('user')):
    print('ok')
else:
    print('not')

not


In [401]:
None==soup.find('p', class_='see_all').find(href=re.compile('user-reviews'))
# soup.find_all(href=re.compile('critic-reviews'))

True

In [ ]:
soup.find('ol', class_='reviews user_reviews')

In [174]:
import os
os.open('ding.mp3')

TypeError: open() missing required argument 'flags' (pos 2)

In [175]:
playsound('ding.mp3')


    Error 263 for command:
        open ding.mp3
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close ding.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: ding.mp3


PlaysoundException: 
    Error 263 for command:
        open ding.mp3
    The specified device is not open or is not recognized by MCI.

In [155]:
from pydub import AudioSegment
from pydub.playback import play
  
# for playing mp3 file
song = AudioSegment.from_mp3("C:/Users/trent/Desktop/VGR/dinger.mp3")
play(song)

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [369]:
# def get_reviews_per_game(startpage_link, next_page, end_page, game_num_start, game_num_end = 101): 
#     next_page = next_page
#     print(f'parsing page: {next_page-1}/{end_page}')
    
#     try:
#         response = requests.get(startpage_link, headers = user_agent)
#         soup = bs(response.text, 'html.parser')
#         current_page = int(soup.find('span', class_='page_num').text)
#     except:
#         playsound('ding.mp3')
    
# #     Loop through current page and extract links to game pages
#     game_num = game_num_start
#     for game in soup.find_all('a', attrs = {'class':'title'})[game_num - 1:game_num_end]:
#         game_link = url + game.get('href')
        
# #         Connects to game page summary
#         sleep(3)
#         try:
#             game_response = requests.get(game_link, headers = user_agent)
#             game_soup = bs(game_response.text, 'html.parser')
#         except:
#             playsound('ding.mp3')
        
# #         testing
        
        
#         count = 1
#         while game_soup.a==None and count<2:
#             print('Error, Trying GAME Again')
#             sleep(3)
#             game_response = requests.get(game_link, headers = user_agent)
#             game_soup = bs(game_response.text, 'html.parser')
#             count += 1
#             print('Still None?:', game_soup.a==None)
#             print('response code:', game_response.status_code)
        
#         if game_soup.a==None:
#             sleep(3)
#             game_response = requests.get(game_link+'/robots.text', headers = user_agent)
#             game_soup = bs(details_response.text, 'html.parser')
#             if game_soup.find('a')==None:
#                 print(f'Game {game_num} GAME Failed')
#                 playsound('ding.mp3')
#                 return
            
            
# #         get number of reviews
#         try:
#             if 
        
#             num_games['num_games'].append(int(game_soup.find_all('p', class_='see_all')[1].text.split()[2]))
#             num_games['title'].append(game_soup.find('h1').text)    
            
            
#         print(f'Game {game_num} completed')
#         game_num +=1
        
#     if next_page == (end_page+1):
#         find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
#         startpage_link = url + find_next_page.get('href')
#         print(f'Done, Next Start Page: {startpage_link}')
#         print('Copy and Paste the link to start_page or use startpage_link variable')
#         playsound('ding.mp3')
#         return startpage_link
#     if next_page>(end_page+1):
#         print('Increase End page')
#         playsound('ding.mp3')
#         return
#     if current_page>next_page:
#         print('Increase Next page')
#         playsound('ding.mp3')
#         return
#     sleep(3)
    
# #     Recurse through the following pages
#     find_next_page = soup.find('a', class_='page_num', string = f'{next_page}')
#     nextpage_link = url + find_next_page.get('href')
#     next_page += 1
#     get_reviews_per_game(nextpage_link, next_page, end_page, game_num_start = 1, game_num_end = 101)